# Introduction to Regression & Classification in Python

# Session Overview

Now that you've had some experience using R, we're going to move over to Python to look at **Machine Learning** tasks.

We're going to get some practice for two common types of machine learning tasks:

1. Regression 
2. Classification 

### Why Python?
The content in today's session could be accomplished in either R or Python. Python is more widely used for machine learning tasks, especially when considering neural networks (will be discussed in the following lectures). We are introducing today's material in Python to give a flavor of a different programming language, including its syntax and functionality. 

# Part 0: Preparing Data

Some words about the packages:

**scikit-learn**:
- A popular machine learning library for Python
- Includes a wide variety of models, including linear and logistic regression, k-means clustering, and support vector machines.
- Has a consistent interface to model fitting, prediction, and evaluation
- Also has built-in functions for data preprocessing, such as feature scaling and dimensionality reduction

**pandas**:
- A powerful library for data manipulation and analysis
- Provides data structures, such as Series and **DataFrame**, that allow for easy manipulation of data
- Built-in functions for data cleaning, filtering, and manipulation, making it a powerful tool for data preprocessing
- Strong integration with other popular libraries, such as NumPy and Matplotlib, for data visualization and analysis

**numpy**:
- A library for numerical computing in Python
- Provides a powerful array object and a large collection of mathematical functions
- Often used as the base array structure for other libraries such as Pandas and scikit-learn

**seaborn**:
- A library for data visualization in Python
- Built on top of Matplotlib
- Provides a high-level interface for creating informative and attractive statistical graphics
- Specifically designed for visualizing statistical models and their results.

In [ ]:
import pandas as pd # Pandas 
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# surpress warnings
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
## Load the data
listings_raw = pd.read_csv('./data/listings_with_amenities.csv')

### A little bit of data exploration and cleaning

In [ ]:
## Explore the first 5 rows
listings_raw.head(5)

In [ ]:
## Display the columns names
print(listings_raw.columns)

Before moving forward, let's do some basic data cleaning.

In [ ]:
def clean_price(p):
    p = p.replace('$','').replace(',','') # replace all '$' and ',' and cast to float 
    return float(p)

# We will "apply" this function to each element of our price column, and replace the price column with these values
listings_raw.loc[:,'price'] = listings_raw.loc[:,'price'].apply(lambda x: clean_price(x))
listings_raw.loc[:,'price']

# Alternatively
# listings_raw["price"] = pd.to_numeric(listings_raw["price"].str.replace("$","").str.replace(",",""))


Let's observe the price distribution using seaborn (more details below):

In [ ]:
sns.distplot(listings_raw["price"], kde=True, color='b')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Distribution of Prices')


# Alternatively: 
# sns.distplot(listings_raw["price"])

In [ ]:
# We'll also clean up some outliers
listings_clean = listings_raw.query('accommodates <= 10 & price <= 1000')
listings_clean = listings_clean.query('maximum_nights <= 2000')

# ALTERNATIVELY:
# listings_clean = listings_raw[(listings_raw["accommodates"] <= 10) & (listings_raw["price"] <= 1000) & (listings_raw["maximum_nights"] <= 2000)]

In [ ]:
# Filter by property type
listings_clean = listings_clean[listings_clean['property_type'].isin(["Apartment", "House", "Bed & Breakfast", "Condominium", "Loft", "Townhouse"])]
listings_clean = listings_clean[~listings_clean['neighbourhood_cleansed'].isin(["Leather District", "Longwood Medical Area"])]

# ALTERNATIVELY (You can actually use query):
    # property_types = ["Apartment", "House", "Bed & Breakfast", "Condominium", "Loft", "Townhouse"]
    # neighbourhoods = ["Leather District", "Longwood Medical Area"]
    # listings_raw = listings_raw.query("property_type in @property_types and neighbourhood_cleansed not in @neighbourhoods")

# See the frequencies of each category of property_type
print(listings_raw["property_type"].value_counts())
print()
print(listings_clean["property_type"].value_counts())

In [ ]:
# Find columns with sufficient fill (at most 25% missing values)
cols_keep = listings_clean.columns[listings_clean.isnull().mean() <= 0.25]
listings_clean = listings_clean.loc[:,cols_keep]

# Alternatively
    # listings_clean = listings_raw.dropna(thresh=len(listings_raw)*0.25, axis=1)

In [ ]:
# Let's one-hot encode our property types/categorical variables
listings_clean = pd.get_dummies(listings_clean, columns = ['property_type'])


In [ ]:
# Lets take a look at the dummy variables btw
property_type_columns = listings_clean[[col for col in listings_clean.columns if "property_type" in col]]
property_type_columns.head(5)

In [ ]:
# We're also going to filter to numeric columns for now- 
# You can use one-hot encoding to handle categorical variables, but today we'll stick to this.
listings_clean = listings_clean.select_dtypes(include=np.number)

In [ ]:
# print important statistics
listings_clean.describe()

### Back to the Sea(born)! 

Check out for more ideas there: https://seaborn.pydata.org/introduction.html

Here are some examples for our dataset.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a 2x2 subplot
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# Create the first distribution plot in the first subplot
sns.distplot(listings_clean['price'], ax=ax[0, 0])
ax[0, 0].set_title("Price Distribution")

# Create the second distribution plot in the second subplot
sns.distplot(listings_clean['accommodates'], ax=ax[0, 1])
ax[0, 1].set_title("Accommodates Distribution")

# Create the third distribution plot in the third subplot
sns.distplot(listings_clean['number_of_reviews'], ax=ax[1, 0])
ax[1, 0].set_title("Number of Reviews Distribution")

# Create the fourth distribution plot in the fourth subplot
sns.distplot(listings_clean['review_scores_rating'].dropna(), ax=ax[1, 1])


##### Alternatively - use for loop

In [ ]:
# Create a 2x2 subplot
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# List of columns
columns = ['price', 'accommodates', 'number_of_reviews', 'review_scores_rating']
numeric = listings_clean[columns]

# Loop through the columns and create the distribution plots
for i, column in enumerate(columns):
    sns.distplot(listings_clean[column].dropna(), ax=ax[i//2, i%2])
    ax[i//2, i%2].set_title(f"{column} Distribution")

In [ ]:
# create box plots for outlier detection for numerical data
numeric = listings_clean[['price', 'accommodates', 'number_of_reviews']]
fig, axes = plt.subplots(nrows=len(numeric.columns), ncols=1, figsize=(5,20))
for i, column in enumerate(numeric.columns):
    sns.boxplot(data=listings_clean,y=column,orient="v",ax=axes[i],palette="Greens")

In [ ]:
sns.catplot(data=listings_clean[['property_type_House', 'accommodates', 'price']], kind="swarm", x="accommodates", y="price", hue="property_type_House")

### Missing Data

It is important to check for missing data and handle it properly.

In [ ]:
# List of values to be replaced with np.na
replace_values = ['', ' ', '#VALUE!', 'NA', 'N/A', 'nan', 'NaN']
listings_clean = listings_clean.replace(replace_values, np.nan)

In [ ]:
# Where are there missing values?
missing_vals = listings_clean.isna().sum()
missing_vals.index[missing_vals>0]

One cool way to visualize missing values is by using the missingno library

In [ ]:
import missingno as ms
mv_matrix = ms.matrix(listings_clean,figsize=(40, 10), color = (0.1, 0.4, 0.1), labels=True) 

Let's impute the missing data - we will simply impute the mean. There are other options available in `sklearn.impute`.
Notice that the missingness is in numeric columns only, which makes it easier to deal with. If there are categoric columns with missing data, different approaches need to be taken (e.g. mode imputation).

In [ ]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(listings_clean)

# Get the imputed values and convert back into a dataframe (it will return a matrix)
listings = pd.DataFrame(imp_mean.transform(listings_clean), 
                        columns = listings_clean.columns)

In [ ]:
# ALTERNATIVELY, SINCE MEAN IS A SIMPLE STRATEGY:
    # listings = listings_clean.fillna(listings_clean.mean())

**Please note that imputing missing values with the mean value can change the distribution of the data and can affect the results of your analysis, it's important to keep this in mind when choosing the imputation method.**


Let's see what our feature space looks like now.  

In [ ]:
ms.matrix(listings,figsize=(40, 10), color = (0.1, 0.4, 0.1), labels=True) 

# Part 1: Regression

**In simple terms**

Regression is a statistical method that is used to find the relationship between a dependent variable (also called the outcome or response variable) and one or more independent variables (also called predictors or explanatory variables).

In simple linear regression, there is only one independent variable, and the relationship between the independent variable and the dependent variable is represented by a straight line. The equation of the line is:

$y = b_0 + b_1x$

Where:

- y is the dependent variable
- x is the independent variable
- $b_0$ is the y-intercept of the line (the value of y when x = 0)
- $b_1$ is the slope of the line (the change in y for a unit change in x)

In multiple linear regression, there are two or more independent variables, and the relationship between the independent variables and the dependent variable is represented by a hyperplane. The equation of the hyperplane is:

$y = b_0 + b_1x_1 + b_2x_2 + ... + b_nx_n$

Where:

- y is the dependent variable
- $x_1, x_2, ..., x_n$ are the independent variables
- $b_0, b_1, b_2, ..., b_n$ are the coefficients of the model

The goal of regression is to find the best-fitting line or hyperplane that describes the relationship between the dependent and independent variables. The best-fitting line or hyperplane is the one that minimizes the sum of the squared differences between the predicted values and the actual values.

**We're going to start by solving a regression problem.**

**Inputs**

Regression is a supervised learning problem, which means that we have access to a dataset $\mathcal{D} = \{ (x_i, y_i) \}_{i = 1}^{n}$ where each $x_i \in \mathbb{R}^d$ and each $y_i \in \mathbb{R}$.

Note that for regression, the output, $y$, is **continuous**. If the output is discrete, such as whether or not an apartment listing will be rented (yes/no) or what town the unit is listed in (multiple classes: Boston vs. Cambridge vs. Somerville), we would use a *classification* algorithm. We will consider this in part 2 of the session.
 
**Aim**

The aim of a regression task is to find a function $h: \mathbb{R}^d \to \mathbb{R}$ which allows us to compute an accurate prediction of the output, $y$, for any given input, $x$. These new $x$ and $y$ are *unseen*: that is, we assume they are identically distributed relative to the points in $\mathcal{D}$, but independent.

Exactly what we mean by *accuracy* will be discussed when we talk about model evaluation.

The way we approach the problem of finding $h$ is by selecting a parameterised class of models, and solving an optimization problem to find the best model within this class according to a *loss function*. We usually repeat this process for several model classes.

### Defining our Regression Task

We're going to use the Boston Airbnb dataset, where each entry is a property listing, and aim to use the input variables available to us in order to predict the price of a listing.

Note here that we haven't specified exactly which variables in the dataset will be used to define each $x_i$. This choice of **feature space** is part of the modeling process, and we will experiment with it.

### Ordinary Least Squares

The first class of models we're going to try is linear models. This means we hypothesise that the output, $y$, can be described **using a linear combination of the inputs**, $x$: $h(x) = \beta^{\intercal} x$.

#### Fitting an OLS Model 

Now, we need to choose exactly which inputs will be used for fitting the model. 

For a very simple model, let's just choose `accommodates` as the only input. To fit this model we will use linear regression in scikit-learn:

In [ ]:
from sklearn.linear_model import LinearRegression

X = listings[['accommodates']]
y = listings['price']

# Select the model type
model = LinearRegression()

# Fit the model to our data
model.fit(X, y)

We've trained our first model! Let's look at the model output. You can see the elements of the `model` object by typing `model.` and pressing "tab."

In [ ]:
print(model.coef_)
print(model.intercept_)

Let's look at the 'coefficients' section. In the 'estimate' column, we see that the point estimates for the model coefficients say that the price estimate is \\$53.67 + \\$38.18*(# of people accommodated). 

To visualise the model, let's plot the fitted line. We'll use `matplotlib` for our plotting.

In [ ]:
# Start with a scatterplot of the data
plt.plot(X, y, 'o')

# Add in the regression line
y_pred = model.predict(X)
plt.plot(X, y_pred)
plt.title('OLS Model Fit')
plt.xlabel("Accomodates")
plt.ylabel("Price")

Let's look the model residuals for each accommodation size to see how appropriate the linear model is for this problem. 


### Evaluation

**Mean Squared Error (MSE)**

Mean Squared Error (MSE) is a measure of the average squared difference between the predicted and actual values. It is calculated using the following equation:

$MSE = \frac{1}{n}\sum_{i=1}^n (y_i - \hat{y_i})^2$

Where:

- $y_i$ is the actual value of the dependent variable
- $\hat{y_i}$ is the predicted value of the dependent variable
- $\bar{y_i}$ is the mean of the dependent variable
- n is the number of observations

**R-squared ($R^2$)**

R-squared is a statistical measure of how close the data are to the fitted regression line. It is a value between 0 and 1, where 1 represents a perfect fit, and 0 represents no fit. R-squared is calculated using the following equation.

It represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.  It is defined as:

$R^2 =1- \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum_{i=1}^n(y_i-\hat{y}_i)^2}{\sum_{i}(y_i-\bar{y})^2}$



Intuitively, R2 measures how accurate our model predictions are in comparison to a **baseline model** which
predicts the **mean of y** from the **training set**.



**Mean squared error (MSE) Calculation**

In [ ]:
resid = (y - y_pred)
mse = np.mean(resid**2)
mse

We can also calculate the RMSE, (root) MSE, from a model applied to a dataset:

In [ ]:
rmse = np.sqrt(np.mean(resid**2))
rmse

Scikit-learn has built-in evaluation functions that can do this for you. We'll also print the results out using Python's number formatting to show the MSE/RMSE rounded to two decimals.

In [ ]:
from sklearn import metrics
mse = metrics.mean_squared_error(y, y_pred)
rmse = metrics.mean_squared_error(y, y_pred)**(1/2)
print("MSE = %.2f" % mse)
print("RMSE = %.2f" % rmse)

Clearly this measure is highly affected by the scale of the data. We can also use the 'R squared' coefficient, which is more interepretable since it is on a standardized scale.

**R-squared calculation**

In [ ]:
def r_squared(y_true, y_pred, y_mean):
    ss_res = ((y_true - y_pred)**2).sum()
    ss_tot = ((y_true - y_mean)**2).sum()
    return (1 - (ss_res/ss_tot))

r2 = r_squared(y, y_pred, np.mean(y))
print("R^2 = %.2f" % r2)


The R squared value is what we will use to evaluate the performance of our models in this session. But, it's important to note that this is definitely not the only choice we could have made! Check out scikit-learn's full library of [evaluation metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) to see what else is out there.

**Question**: What is the relationship between the **loss function** and the **measure of accuracy** used to evaluate a model? How do we know which is the best choice for each? 

#### Answer
- The loss function is used to fit a model, and the measure of accuracy is used to evaluate how well it explains the data. 
- The measure of accuracy usually comes from the task we aim to solve, and the loss function is often something that we will play around with until we find a good model.

#### Training, Validation and Testing Splits

Recall that our ultimate goal in this supervised learning task is to be able to predict price ($y$) from an **unseen** set of inputs ($x$, although we are still playing around with the input variables which define it).

When building the OLS model, we used the entire dataset. Simply taking the R squared value on this dataset as a measure of performance is clearly not fair -- since we want the model to generalise to unseen data.

To address this problem, we often split the dataset into three different chunks:

1. **Training data**: the data we build our models on.
2. **Validation data**: the data we tune hyperparameters on.
3. **Testing data**: the data we use to obtain a final estimate of performance.

The validation set is like an 'intermediate' estimate of performance. If we didn't use the validation set for this purpose, the only way of selecting the best model from a model class would be to look at its performance on the training set or the testing set.

`train_test_split` in scikit-learn provides an easy way of creating these data partitions. Let's start by just creating training/testing sets. We'll get to parameter tuning in a minute.

In [ ]:
from sklearn.model_selection import train_test_split

features =  listings.columns.drop(['id', 'scrape_id', 'host_id', 'latitude', 'longitude', 
                                   'availability_30','availability_60', 'availability_90', 'availability_365',
                                   'host_total_listings_count','calculated_host_listings_count',
                                   'price'])

X_train, X_test, y_train, y_test = train_test_split(listings[features], listings['price'], 
                                                    train_size = 0.75, random_state = 1)

print("Training Size = %d" % X_train.shape[0]) # To print integers, we can pass '%d' into our print statement 
print("Testing Size = %d" % X_test.shape[0])

Let's now observe our training data and do some basic pre-processing before training the models.

In [ ]:
# Describe the data 
X_train.describe()

In [ ]:
# We should normalize or standardize our data, so that features are on the same scale.

### The "manual" way ###

# Normalization
X_train_normalized = (X_train - X_train.min())/(X_train.max()-X_train.min())
X_test_normalized = (X_test - X_train.min())/(X_train.max()-X_train.min())

# Standardization
X_train_standardized = (X_train - X_train.mean())/X_train.std()
X_test_normalized = (X_test - X_train.mean())/X_train.std()

In [ ]:
### The "package" way ###

# Normalization
from sklearn.preprocessing import MinMaxScaler

norm_scaler = MinMaxScaler()
norm_scaler.fit(X_train)
X_train_normalized2 = norm_scaler.transform(X_train)
X_test_normalized2 = norm_scaler.transform(X_test)


# Standardization
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_standardized2 = std_scaler.transform(X_train)
X_test_standardized2 = std_scaler.transform(X_test)

# You can look at the mean and std from the scaler
# print(std_scaler.mean_)
# print(std_scaler.scale_)


### Model Iteration

Now that we're equipped to build models and evaluate their performance, let's start iterating to find better models.

We've glossed over the precise choice of variables to use in order to explain price, so let's try a few different combinations. 

Now we'll write a function that accepts data (X,y) and splits into training/testing data, fits a model, and returns the R squared value on each partition.

In [ ]:
def evaluate(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    rmse_train = metrics.mean_squared_error(y_train, y_train_pred)**(1/2)
    rmse_test = metrics.mean_squared_error(y_test, y_test_pred)**(1/2)
    
    print("Train RMSE = %.2f" % rmse_train)
    print("Test RMSE = %.2f" % rmse_test)
    
    rsq_train = r_squared(y_train, y_train_pred, np.mean(y_train))
    rsq_test = r_squared(y_test, y_test_pred, np.mean(y_train))
    
    print("Train R^2 = %.2f" % rsq_train)
    print("Test R^2 = %.2f" % rsq_test)

def train_eval(model, X, y, split_seed = 1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = split_seed)
    
    # we will perform Min-Max Scaling
    norm_scaler = MinMaxScaler()
    norm_scaler.fit(X_train)
    X_train_normalized2 = norm_scaler.transform(X_train)
    X_test_normalized2 = norm_scaler.transform(X_test)
    
    model = model.fit(X_train, y_train)
    evaluate(model, X_train, y_train, X_test, y_test)

In [ ]:
print("Price ~ Accomodates")
train_eval(LinearRegression(), listings[['accommodates']], listings['price'])
print("\nPrice ~ Accomodates + Reviews/Month")
train_eval(LinearRegression(), listings[['accommodates','reviews_per_month']], listings['price'])
print("\nPrice ~ Accomodates + Reviews/Month + Rating")
train_eval(LinearRegression(), listings[['accommodates','reviews_per_month','review_scores_rating']], listings['price'])
print("\nPrice ~ All Variables")
train_eval(LinearRegression(), listings[features], listings['price'])


**Question**: Can we say anything about these results?

**Answer**: We can be relatively confident that the largest model is the best performing. We also see that the testing performance is significantly worse than the training performance. Usually this indicates overfitting.

### Regularization

In some situations, overfitting is more obvious -- and since it is a common problem when building models, we will now look at how it can be addressed.

Regularization is a tool which helps us to avoid overfitting by penalising model complexity. Mathematically, we add a term to the loss function in the optimization problem to be solved. Recall that the OLS formulation we've worked with is:

$$\min_w \: \frac{1}{n} \sum_{i = 1}^{n} (w^\intercal x_i - y_i)^2$$

With a regularization term, this becomes:

$$\min_w \: \frac{1}{n} \sum_{i = 1}^{n} (w^\intercal x_i - y_i)^2 + \lambda \Omega(w)$$


$\Omega(w)$ is a penalty on the complexity of the model. Two common choices for $\Omega(w)$ are:

1. $\Omega(w) = ||w||_2^2 = \sqrt{\sum_{i=1}^n w_i^2} $: this is Ridge regression.
2. $\Omega(w) = ||w||_1 = \sum_{i=1}^n |w_i|$: this is Lasso regression.

Both types of regression shrink the elements of the optimal $w^*$ vector towards 0 -- but in different ways. We will focus on Lasso -- which tends to shrink the coefficients so that some are equal to 0. Yes, we all know Lasso is not a sparse method, but "often" it leads to sparser solutions.

### Cross-Validation

We need a way of selecting the one with the best parameter for our task.

**Question**: How would we do this using the training/validation/testing splits?

**Answer**: Fit models on the training set, varying lambda. Then obtain an estimate of performance for each model on the validation set, and choose the best. Obtain a final estimate of performance for this model on the testing set.

Here, because `GridSearchCV` makes it easy, we're going to use a similar technique called cross-validation. Generally, we only consider this necessary when we're worried we have too little training data to obtain an accurate estimate of validation performance.

The idea behind cross-validation is: repeating the training/validation process multiple times provides us with several estimates of validation performance. Taking the average of these hopefully gives us an estimate which is less affected by noise.

To cross-validate, we start with only two partitions of the dataset:

1. Combined training/validation set: the data that we repeatedly train and validate on.
2. Testing set: the data we obtain our final performance estimate on.

But, how do we train and validate?

- First, select a number of *folds*.
- Then divide the training/validation data into this number of equal-sized partitions.
- For each fold, repeat the training/validation procedure. The fold is the validation data, and the other folds are training data.
- Average the performance of each model across folds and pick the hyperparameters which produce the best model.
- Fit a model on the entire training set using the selected values of hyperparameters.



![](https://miro.medium.com/max/1838/1*AAwIlHM8TpAVe4l2FihNUQ.png)

We'll use our training set from before and run 5-fold cross-validation to determine the best `alpha`.

**Question**: 
- Why is validation an important step before testing? 
- When selecting the best model based on validation, how do we expect the test evaluation to behave?


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Define the grid that we want to search over: np.arange allows you to create a list of values you want to cross-validate
grid = {'alpha':np.arange(0.01, 1, 0.01)}

# Define the parameters for the model 
gs = GridSearchCV(
    estimator = Lasso(max_iter=1000000), 
    param_grid = grid, 
    cv=5)

## Fit the model
random.seed(1)
gs.fit(X_train, y_train)
m = gs.best_estimator_

## Print the best parameters determined by the cross-validation
print("Alpha: %.3f" % gs.best_params_['alpha'])

In [ ]:
coef_output = pd.DataFrame({'feature':X_train.columns,
              'coefficient':m.coef_})
print("Number of zeros: %d" % (coef_output.query('coefficient == 0').shape[0]))
coef_output.sort_values('coefficient')

We still haven't obtained an error measurement on the testing data, which will allow us to compare our best regularised model with the unregularised one. 

Let's generate predictions and calculate our performance metrics:

In [ ]:
evaluate(gs, X_train, y_train, X_test, y_test)

Let's see how well we do if we use a higher alpha parameter, even though its slightly suboptimal.

In [ ]:
clf = Lasso(alpha=0.5)
clf.fit(X_train, y_train)
evaluate(clf, X_train, y_train, X_test, y_test)
print("Number of zeros: %d" % sum(clf.coef_ == 0))

The previous best model had a test RMSE of 82.64 (and $R^2$ of 0.41), so we now have a slightly better model. Regularization has also given us a way of obtaining an accurate model with fewer nonzero coefficients. Even when taking the higher suboptimal alpha, we still do nearly as well with even higher interpretability (5 fewer nonzeros than the optimal Lasso model). 

## Other Regressors!

Until now we have worked on Linear Regressors. Let's expand see how other tree based models perform!

### Decision Trees & Random Forests


### Decision Trees 
We will briefly explore regression trees (often referred to as CART, for Classification And Regression Trees).

A tree makes predictions by grouping similar observations and then averaging the observations or using a linear regression model for the group. Groups can be thought of as nodes on a tree, and tree branches correspond to logical criteria on the predictor variables. There's a lot of neat math that goes into building the trees, but we won't get into that today. For now let's get familiarized by looking at a simple example. We will use the `DecisionTreeRegressor` in scikit-learn.	

#### Tuning the CART model
If we want to construct high accuracy decision tree models, then we need to tune the parameters.  CART has many parameters that specify how the decision tree is constructed. Today we'll focus on `max_depth`. 
* If the tree is too big => too many splits => we have an over-fitting problem.
* If the tree is too small => too few splits => we have an under-fitting problem.

We want to find a tree in the middle, that is "just right". We'll use `GridSearchCV` package to tune the tree depth using cross-validation. 

The model construction step follows the same established pattern.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

params = {
    'max_depth' : [i for i in range(1,10,1)]
}

model = GridSearchCV(
        DecisionTreeRegressor(),
        params,
        cv=5,
        verbose=1
)

model.fit(X_train, y_train)
evaluate(model, X_train, y_train, X_test, y_test)

In [ ]:
# lets plot the tree
from sklearn.tree import plot_tree

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (50,10), dpi=200)
plot_tree(model.best_estimator_, feature_names = X_train.columns, filled = False, fontsize = 8);
fig.savefig('cart_tree.png')

### Random Forests

We will briefly take a look at random forests, using the [RandomForestRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). A random forest is a collection of slightly randomized decision trees (hence the name "forest"), and can be used for classification or regression. They often have excellent predictive performance, but can be expensive to train and lack interpretability. Random forests have many hyperparameters that can be tuned to achieve the best possible predictive performance. Perhaps the most important hyperparameter is the number of trees to include in the forest. More trees results in a longer training time but can improve prediction and decrease overfitting.
The goal of Random Forest is to train a pool of base tree learners with a high variance but a small bias. They are not necessarily weak learners!
With bagging, we explicitly train weaker (but not necessarily weak) learners.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

params = {
    'n_estimators' : [100, 200],
    'max_depth' : [i for i in range(1,10,2)]
}

model = GridSearchCV(
        RandomForestRegressor(random_state=42),
        params,
        cv=5,
        verbose=1
)

model.fit(X_train, y_train)
evaluate(model, X_train, y_train, X_test, y_test)

### Boosted Trees

Boosting means combining a learning algorithm in series to achieve a strong learner from many sequentially connected weak learners. In case of gradient boosted decision trees algorithm, the weak learners are decision trees.

Each tree attempts to minimize the errors of the previous trees. Trees in boosting are weak learners but adding many trees in series and each focusing on the errors from previous one make boosting a highly efficient and accurate model.

A major difference between random forests and boosting algorithms is the overall size of the trees. In a random forest, we tend to choose trees as strong learners — they would do just fine as a decision tree on their own. In boosting algorithms, trees are artificially limited to very shallow depths (in practice rarely more than depth 5), to ensure that each model is only slightly better than random chance.

Let' work with the famous XGBoost, maybe the most widely used machine learning model!

In [ ]:
from xgboost.sklearn import XGBRegressor
params = {
    'n_estimators' : [50, 100],
    'max_depth' : [2, 4, 6, 8, 10]
}

model = GridSearchCV(
        XGBRegressor(random_state=42),
        params,
        cv=5,
        verbose=1
)

model.fit(X_train, y_train)
evaluate(model, X_train, y_train, X_test, y_test)

![](https://www.datascienceland.com/media/uploads/2020/11/29/difference.png)

# Part 2: Classification
So far we've looked at models which predict a continuous response variable. There are many related models which predict categorical outcomes, such as whether an email is spam or not, or which digit a handwritten number is. We'll take a look at some of these. Let's begin with Logistic Regression!

### Logistic Regression
Logistic regression is part of the class of generalized linear models (GLMs), which build directly on top of linear regression. These models take the linear fit and map it through a non-linear function. 

Specifically, logistic regression is used when the dependent variable is **binary** (e.g. true/false, yes/no, 0/1) and the goal is to find the relationship between one or more independent variables and the probability of the dependent variable being true or 1.

For logistic regression, the function $f(\cdot)$ is given by $\displaystyle f(z) = \frac{1}{1+\exp(-z)}$ (the *logistic function* - also called the *sigmoid function*).


Where:

- $f(z)$ is the probability of the dependent variable being true or 1
- z is the linear combination of the independent variables and the coefficients of the model: 
$\displaystyle z = b_0 + b_1x_1 + b_2x_2 + ... + b_nx_n$




Let's apply a logistic regression model to the `listings` data. Let's try to predict which listings have elevators in the building by using `price` as a predictor. Remember that our `amenity_Elevator_in_Building` variable already stores our binary outcome of interest (1 = elevator, 0 = no elevator).

We need to re-split our data with y as the `amenity_Elevator_in_Building` variable. We will also *stratify* our split on this variable, meaning that the data is split to preserve the ratio of units with elevators in both the training and testing sets.

In [ ]:
features =  listings.columns.drop(['id', 'scrape_id', 'host_id', 'latitude', 'longitude', 
                                   'host_total_listings_count','calculated_host_listings_count',
                                   'availability_30','availability_60', 'availability_90', 'availability_365',
                                   'amenity_Elevator_in_Building'])
X_train, X_test, y_train, y_test = train_test_split(listings[features], listings['amenity_Elevator_in_Building'], 
                                                    train_size = 0.7, random_state = 42,
                                                   stratify = listings['amenity_Elevator_in_Building'])


Instead of linear regression, we'll now use `LogisticRegression()`, but the syntax is very similar. We'll skip ahead directly to using a grid search to tune regularization parameters. As you can see, the same steps apply even with a new model:
1. Define a parameter grid
2. Initialize a GridSearchCV object with the tuning grid and other fixed parameters
3. Fit the model!

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Define the grid that we want to search over
param_grid = {'C': np.arange(0.001, 1, 0.05), 
              'penalty': ['l2','l1'], 
              'solver': ['liblinear']}

# Define the parameters for the model 
gs = GridSearchCV(LogisticRegression(random_state=42, max_iter = 1000),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)
## Fit the model
random.seed(42)
gs.fit(X_train, y_train)
m_lr = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

In [ ]:
# Let's look at the coefficients
coef_output = pd.DataFrame({'feature':X_train.columns,
              'coefficient':m_lr.coef_[0]})

print("Number of zeros: %d" % (coef_output.query('coefficient == 0').shape[0]))
coef_output.sort_values('coefficient')

We can now explore out-of-sample performance. Ultimately, we want to predict whether or not a listing has an elevator. However, logistic regression gives us something a bit different: a probability that each listing has an elevator. This gives us flexibility in the way we predict. The most natural thing would be to predict that any listing with predicted probability above 0.5 *has* an elevator, and any listing with predicted probability below 0.5 *does not have* an elevator. But what if I use a wheelchair and I want to be really confident that there's going to be an elevator? I may want to use a cutoff value of 0.9 rather than 0.5. In fact, we could choose any cutoff value and have a corresponding prediction model.	

This is where AUC comes in. For every cutoff, we'll plot the *false positive rate* against the *true positive rate* and then take the area under this curve.	

In [ ]:
train_pred = m_lr.predict_proba(X_train)[:,1]
test_pred = m_lr.predict_proba(X_test)[:,1]

print("Train AUC: %.3f" % metrics.roc_auc_score(y_train, train_pred))
print("Test AUC: %.3f" % metrics.roc_auc_score(y_test, test_pred))

train_fpr, train_tpr, _ = metrics.roc_curve(y_train, train_pred)
test_fpr, test_tpr, _ = metrics.roc_curve(y_test, test_pred)

# plot the roc curve for the model
plt.plot(train_fpr, train_tpr, linestyle='--', label='Train')
plt.plot(test_fpr, test_tpr, marker='.', label='Test')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

We can also look at threshold-based metrics. By selecting a decision threshold $t$, we predict all observations with a predicted probability $\hat{y} \geq t$ to have an elevator and all others to have no elevator. 

Let's calculate the training and testing accuracy for our model:

In [ ]:
t = 0.5
print("Train Accuracy: %.3f" %  metrics.accuracy_score(y_train, train_pred > t))
print("Test Accuracy: %.3f" %  metrics.accuracy_score(y_test, test_pred > t))

# If we wanted to get all of the elements of the confusion matrix, we could pull them like this:
tn, fp, fn, tp = metrics.confusion_matrix(y_train, train_pred > t).ravel()

**Question:** What is the baseline accuracy for this prediction task?

**Answer:** The training set has 591/2449 = 24.1% elevators. A naive baseline model would predict the most common class, "no elevators" for all observations. This would be correct 75.9\% of the time on the training set. The testing set also has elevators in 24.2\% of units, so it would have 75.8\% accuracy. (Remember that we stratified our sample, so it is not coincidental that the percentages are the same!).

In [ ]:
np.mean(1-y_train)

Before we move on, we'll wrap all of our evaluation steps into a function that we can use to evaluate other models in a consistent way.

In [ ]:
def classification_evaluation(model, X_train, y_train, X_test, y_test, t = 0.5):
    train_pred = model.predict_proba(X_train)[:,1]
    test_pred = model.predict_proba(X_test)[:,1]

    print("Train AUC: %.3f" % metrics.roc_auc_score(y_train, train_pred))
    print("Test AUC: %.3f" % metrics.roc_auc_score(y_test, test_pred))
    
    print("\nTrain Accuracy: %.3f" %  metrics.accuracy_score(y_train, train_pred > t))
    print("Test Accuracy: %.3f" %  metrics.accuracy_score(y_test, test_pred > t))

    train_fpr, train_tpr, _ = metrics.roc_curve(y_train, train_pred)
    test_fpr, test_tpr, _ = metrics.roc_curve(y_test, test_pred)

    # plot the roc curve for the model
    plt.plot(train_fpr, train_tpr, linestyle='--', label='Train')
    plt.plot(test_fpr, test_tpr, marker='.', label='Test')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [ ]:
classification_evaluation(m_lr, X_train, y_train, X_test, y_test)

As you can see, `sklearn.metrics` is versatile and allows you to calculate and plot a bunch of different performance metrics. 


### Classification Trees 
We will briefly explore classification trees.

A (binary) classification tree makes predictions by grouping similar observations and then assigning a probability to each group using the proportion of observations within that group that belong to the positive class. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Define the grid that we want to search over
param_grid = {"max_depth": np.arange(3,10,1), "criterion": ['gini', 'entropy']}

# Define the parameters for the model 
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)

## Fit the model
random.seed(1)
gs.fit(X_train, y_train)
m_cart = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

We can plot and save the resulting tree as follows:	

In [ ]:
from sklearn.tree import plot_tree

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (50,10), dpi=200)
plot_tree(m_cart, feature_names = X_train.columns, filled = False, fontsize = 8);
fig.savefig('cart_tree_clf.png')

Let's use our evaluation function from before to see how well this model performs! 

In [ ]:
classification_evaluation(m_cart, X_train, y_train, X_test, y_test)

The CART model does worse than the logistic regression model, with an AUC of 0.870 (vs. 0.9) on the test set.

### Random Forests


Let's start by training a random forest model for a classification task. We will perform the same task of predicting whether or not a listing has an elevator, using price and neighborhood as predictors. We will compare the performance of random forest to what we got using our simple CART model. We'll use a grid search to tune the number of estimators.

*Note: This will take a while!*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [100,200,300],
     'max_depth': np.arange(3,8,1),
     'max_features': ['auto'],
     'min_samples_leaf': [0.01,0.02],
    'criterion' :['gini']
}
        
# Define the parameters for the model 
gs = GridSearchCV(RandomForestClassifier(random_state=42),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)

## Fit the model
random.seed(1)
gs.fit(X_train, y_train)
m_rf = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

In [ ]:
# Let's evaluate the model performance
classification_evaluation(m_rf, X_train, y_train, X_test, y_test)

In general, random forests give stronger performance than single decision trees, but there is a sacrifice in interpretability. Interestingly, the logistic regression model is slightly better than the random forest model. The prediction accuracy is now much better, and it doesn't look like we are overfitting *too* much. It would likely be even better if the other hyperparameters of the random forest model were properly tuned! 

### Boosted Trees

In [ ]:
import xgboost as xgb
import time

# Define the parameter grid
param_grid = {
    'n_estimators': [100,200],
     'max_depth': np.arange(2,6,1),     
}
        
# Define the parameters for the model 
gs = GridSearchCV(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)

## Fit the model
random.seed(1)
start_time = time.time()
gs.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

m_xgb = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

In [ ]:
classification_evaluation(m_xgb, X_train, y_train, X_test, y_test)

LightGBM, short for Light Gradient Boosting Machine, is another distributed gradient boosting framework originally developed by Microsoft. Light GBM uses leaf-wise splitting over depth-wise splitting which enables it to converge much faster but also leads to a higher tendency to overfit.

In [ ]:
import lightgbm as lgb

# Define the parameter grid
param_grid = {
    'n_estimators': [100,150],
     'max_depth': np.arange(2,6,1),
}
        
# Define the parameters for the model 
gs = GridSearchCV(lgb.LGBMClassifier(random_state=42),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)

## Fit the model
random.seed(1)

start_time = time.time()
gs.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

m_lgb = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

In [ ]:
classification_evaluation(m_lgb, X_train, y_train, X_test, y_test)

### Neural Networks

We won't cover neural networks during this lecture, Angelos and Leo will handle that in detail in the next lectures! Let's a see only a toy example .

In general, we use TensorFlow or Pytorch to implement neural-networks but scikit-learn includes a basic implementation for feedforward neural networks.

In [ ]:
from sklearn.neural_network import MLPClassifier

# Define the parameter grid
param_grid = {
     'hidden_layer_sizes': [(128), (128,64)], # the number of nodes at the each layer
     'alpha': [1e-3, 1e-4] # regularization term, aka penalty term, that combats overfitting
}

#It is very important to scale the data for neural networks.
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_standardized2 = std_scaler.transform(X_train)
X_test_standardized2 = std_scaler.transform(X_test)
        
# Define the parameters for the model 
gs = GridSearchCV(MLPClassifier(random_state=42),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 1, n_jobs = -1)

## Fit the model
random.seed(42)
gs.fit(X_train_standardized2, y_train)
m_mlp = gs.best_estimator_
print("Best parameters: ", gs.best_params_)

In [ ]:
classification_evaluation(m_mlp, X_train_standardized2, y_train, X_test_standardized2, y_test)

# Part 3: Advanced Topics 




## A. Interpretability / Explainability


#### Interpreting Random Forest and Boosted-Tree Models
Although random forest and boosted-tree models are not very interpretable and hard to visualize, there is a popular method called *variable importance* that is commonly used. We will use the built-in `feature_importances_` from these models. This tells us which variables the random forest or boosted-tree model has determined as the most important for predicting whether or not a listing contains an elevator. It uses (usually) Gini importance, which is the mean decrease in node impurity (you can read more about this metric online), to rank the importance of all of the predictors used in the model. There are other metrics besides the Gini Importance that can also be used as a metric.

In [ ]:
#Random Forest
importance_output = pd.DataFrame({'feature':X_train.columns,
                                 'importance':m_rf.feature_importances_}).sort_values('importance', ascending = False)

plt.barh(importance_output['feature'][0:10], importance_output['importance'][0:10])

In [ ]:
#XGBoost
importance_output = pd.DataFrame({'feature':X_train.columns,
                                 'importance':m_xgb.feature_importances_}).sort_values('importance', ascending = False)

plt.barh(importance_output['feature'][0:10], importance_output['importance'][0:10])

In [ ]:
#LightGBM
importance_output = pd.DataFrame({'feature':X_train.columns,
                                 'importance':m_lgb.feature_importances_}).sort_values('importance', ascending = False)

plt.barh(importance_output['feature'][0:10], importance_output['importance'][0:10])

For XGBoost, the variable importance plot shows us that whether or not the unit has a gym, how many listings the host has, and whether there is a doorman are the most important features. 

This not only gives us intuition about our models, but can also be used to select variables to train other models. However, there are some shortcomings. For example, price is important in the LightGBM model, but what is the direction of this relationship? Are high or low prices associated with elevator likelihood?

#### Interpretation with SHapley Additive exPlanations (SHAP)
There has been a lot of research devoted to better methods of interpreting "black box" machine learning models, like random forests. The [SHAP](https://shap.readthedocs.io/en/latest/) package offers an easy-to-use interface that estimates the contribution of each variable to each observation's prediction. This gives insight into both the *magnitude* and *directionality* of a feature's importance. The package also allows for plots of interactions between variables to understand important nonlinearities. We'll do this for the XGBoost model, but we could use the same package for many other models (including neural networks).

In [ ]:
import shap  ## you can install this in terminal: 'conda install -c conda-forge shap'

explainer = shap.TreeExplainer(m_xgb); # data=test_x, model_output="probability");
shap_values = explainer.shap_values(X_train)#[1]; # index 1 pulls the P(y=1) SHAP values for RandomForest.

##### Summary Plot
We'll start with the summary plot, which ranks the features by importance (mean absolute SHAP value) and also shows the direction of the relationship. 

Features are ordered by decreasing significance, with the most important feature listed at the top of the plot. For a given feature, the corresponnding row shows a plot of the feature's impact on the prediction as the value ranges from its lowest (blue) to highest (red) value. Higher SHAP values correspond to increased likelihood of having a positive outcome (i.e. having an elevator). Thus, features with the color scale oriented blue to red (moving left to right) have increasing risk as the feature increases, such as hotel count. Note: the categorical variables are encoded as a binary value (e.g. 0=no gym, 1=gym), so "higher" values correspond to having the indicated feature.

In [ ]:
plt.close()
shap.summary_plot(shap_values, X_train, show=True,
                  max_display=10,
                  plot_type="violin")

The summary plot gives us insight into the key driving features; this is similar to Gini feature importance but gives us more granular insight.
- Gym, Doorman, Wheelchair Accessible,... all increase the probability of an elevator.
- As price increases, the probability of an elevator increases.
- Listings from hosts with more listings have higher probability of an elevator.
- The property type being a house decreases the probability.

##### Dependence Plots

Let's dive into the relationship between price and whether the unit is predicted to have an elevator. We can create a dependence plot or price to see how varied prices affect the output probability.

In [ ]:
plt.close()
shap.dependence_plot('price', shap_values, X_train,
                       interaction_index = None,
                       xmin="percentile(1)", xmax="percentile(99)",
                       dot_size=3,show=False)

It looks like for the listings below 180/night, the price lowers the probability that the unit will have an elevator (SHAP value < 0). Above 180/night, there is a positive impact on probability of having an elevator. Above 300/night or so, however, the impact on the probability stabilizes; the probability does not continue to increase with prices above 300. 

## B. Ensemble Modeling


Our goal is to obtain additional improvement from the top previous models. A very successful technique, widely used in data science competitions, is to build an ensemble model, i.e., using a combination (consensus) of different "base learners" to make a final decision. 

The general principle is to:
- create train/val/test sets. Note: It is important to create a validation set.
- fit a few different models on the Training set, that we evaluate on the Validation set
- train a consensus model using the predictions of these models on the Validation set (which is unseen data) 
- evaluate the performance on the Test set

**Question**: Why is it important to maintain a validation data to build an ensemble model?

In [ ]:
#We keep our train/test set from previously but we separate our train set into train and val.
X_train_ensemble, X_val_ensemble, y_train_ensemble, y_val_ensemble = train_test_split(X_train, y_train, 
                                                    train_size = 0.75, random_state = 6,
                                                   stratify = y_train)

In [ ]:
#We previously trained several models. For prototyping sake, we don't do our validation or cross-validation again, but you always should in practice!

#Get predictions on unseen data
def create_prediction_data(model_list, X_train, y_train, X_val, y_val, X_test, y_test, verbose=False):
    df_val = pd.DataFrame()
    df_test = pd.DataFrame()
    for key in model_list:
        model_list[key].fit(X_train, y_train)
        df_val[key] = model_list[key].predict_proba(X_val)[:,1]
        df_test[key] = model_list[key].predict_proba(X_test)[:,1]
        if verbose:
            print("\n#### " + key +  " ####")
            print("Test AUC: ", metrics.roc_auc_score(y_test, model_list[key].predict_proba(X_test)[:,1]))
            print("Test Acc: ", metrics.accuracy_score(y_test, model_list[key].predict(X_test)))
        #classification_evaluation(model_list[key], X_train, y_train, X_test, y_test)
    return df_val, df_test

In [ ]:
#We get our ensemble data
d_1 = {'logreg': m_lr, 'cart': m_cart, 'rf': m_rf, 'xgb': m_xgb, 'lgb': m_lgb}
d_2 = {'logreg': m_lr, 'rf': m_rf, 'xgb': m_xgb, 'lgb': m_lgb}
d_3 = {'logreg': m_lr, 'rf': m_rf, 'xgb': m_xgb, 'cart': m_cart}

df_val_1, df_test_1 = create_prediction_data(d_1, X_train_ensemble, y_train_ensemble, X_val_ensemble, y_val_ensemble, X_test, y_test, verbose = True)
df_val_2, df_test_2 = create_prediction_data(d_2, X_train_ensemble, y_train_ensemble, X_val_ensemble, y_val_ensemble, X_test, y_test)
df_val_3, df_test_3 = create_prediction_data(d_3, X_train_ensemble, y_train_ensemble, X_val_ensemble, y_val_ensemble, X_test, y_test)

### Average Ensemble

The simplest version of an ensemble model is the average of the base models' probability outputs. Another possibility is to make a "vote" among the base models and use the majority vote as our prediction.

In [ ]:
print("\n#### Probability average with CART included ####")
print("Test AUC: ", metrics.roc_auc_score(y_test, df_test_1.mean(axis = 1)))
print("Test Acc: ", metrics.accuracy_score(y_test, df_test_1.mean(axis = 1) > 0.5))

print("\n#### Probability average without CART ####")
print("Test AUC: ", metrics.roc_auc_score(y_test, df_test_2.mean(axis = 1)))
print("Test Acc: ", metrics.accuracy_score(y_test, df_test_2.mean(axis = 1) > 0.5))

print("\n#### Probability average without LightGBM ####")
print("Test AUC: ", metrics.roc_auc_score(y_test, df_test_3.mean(axis = 1)))
print("Test Acc: ", metrics.accuracy_score(y_test, df_test_3.mean(axis = 1) > 0.5))

### Weighted Average Ensemble

A slightly more sophisticated version of an ensemble model is the weighted average of the base models' probability outputs. We can train a Lasso Logistic Regression to obtain the different weights.

In [ ]:
# Weighted average
# Define the grid that we want to search over
param_grid = {'C': np.arange(0.001, 1, 0.05), 
              'penalty': ['l2','l1'], 
              'solver': ['liblinear']}

# Define the parameters for the model 
gs_ensemble = GridSearchCV(LogisticRegression(random_state=42, max_iter = 1000),
                  return_train_score=True, 
                  param_grid=param_grid, 
                  scoring='roc_auc',
                  cv=5, verbose = 0)
## Fit the model
random.seed(1)
gs_ensemble.fit(df_val_1, y_val_ensemble)
m_lr_ensemble = gs_ensemble.best_estimator_
print("Best parameters: ", gs_ensemble.best_params_)

In [ ]:
print("Test AUC: ", metrics.roc_auc_score(y_test, m_lr_ensemble.predict_proba(df_test_1)[:,1]))
print("Test Acc: ", metrics.accuracy_score(y_test, m_lr_ensemble.predict(df_test_1)))


In [ ]:
# Let's look at the coefficients
coef_output = pd.DataFrame({'feature':df_test_1.columns,
              'coefficient':m_lr_ensemble.coef_[0]})

print("Number of zeros: %d" % (coef_output.query('coefficient == 0').shape[0]))
coef_output.sort_values('coefficient')

The "optimal" weighted average ensemble leads to roughly the same top performance using only 3 out of the 5 models. This is extremely desired for deployment purposes as it means fewer models will need to be implemented which results in a faster and more reliable pipeline.

## C. Automatic ML



### Lazypredict

Lazypredict is a package that trains the most popular machine learning models on your data, with their default hyperparameters values. It can be very useful to find out what are the top performing models before you start yourself the hyperparameter tuning.

I also like to use it to obtain a set of base learners for an ensemble model. I generally keep the 10-15 out-of-the-box top models given by ```lazypredict``` and use a Lasso ensemble afterwards.

In [ ]:
import lazypredict

In [ ]:
#As usual, you can use the Regressor or Classifier version
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

In [ ]:
reg = LazyClassifier(predictions = True, verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train_ensemble, X_val_ensemble, y_train_ensemble, y_val_ensemble)

#Performance is reported on the validation set
models

# Conclusion

### Model Selection
**Question:** Now that we have trained and evaluated so many different models, which one would you select?

**Answer:** It depends on the application, the stakeholders, and the costs of missed positives and negatives (or in continuous prediction, under- and over-predictions). 

In a problem like this, the prediction of whether an Airbnb listing will have an elevator is not highly consequential. However, consider a clinical decision support tool that predicts hospital re-admission or a model that predicts loan defaults. In such cases, there may be a higher premium on interpretability to ensure transparency for patients (or bank clients). 

On the other hand, a tool that is used to triage customer service calls within a company may not need to be as interpretable because it is used purely internally. 

## Take-aways
In this module, we have covered examples of machine learning methods for regression (ordinary and penalized) and classification.  This is just the tip of the iceberg.  There are tons more machine learning methods which can be easily implemented in Python.  

We haven't focused that much on the feature preprocessing and data cleaning (because it is boring?), but this is often one of the most important aspects. 

Remember to always scrutinize your data before doing serious modeling.

In particular: 
- look for outliers, 
- handle missing data (e.g., delete, impute, interpolate),
- observe the feature distributions,
- observe the feature correlations,
- consider engineering a few additional features 
- remember real-world data is always messy...

During modeling:
- always cross-validate or at least validate your hyperparameters,
- do not try to overfit your test set!
- think about deployment: will you have data drifts? Is your model easy to deploy? Does your model run fast?
- choose your evaluation metrics carefully.

To get some extra juice:
- consider using ensemble modeling, but if possible keep it simple
- Automatic ML can save you a lot of time, I wish I knew it before!!

### Notes on sklearn modeling

- There are many, many classification algorithms implemented in `scikit-learn`. [Here's](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) an example that compares a broad set of methods, including SVM, k-Nearest Neighbors, and others.
- XGBoost and LightGBM follow nearly identical syntax to the examples above (using GridSearchCV). However, it is not implemented in scikit-learn. You have to install the [xgboost](https://xgboost.readthedocs.io/en/latest/) library and then adapt the code above to this model.

### Credits

This Notebook is based on previous work by **Holly Wiberg, Léonard Boussioux, and Asterios Tsiourvas**. A **BIG** thank you to all of them!!!